In [8]:
import requests
import json

In [45]:
JSON_INDENT = 2
def format_json(obj):
    """
    Formats a given JSON instance into a readable string with a defined indentation.

    Args:
        obj (Any): The JSON object to format, which should be
            a valid Python dictionary or list structure.

    Returns:
        str: A JSON-formatted string with indentation specified by the global JSON_INDENT.
    """
    try:
        return json.dumps(obj, indent=JSON_INDENT)
    except Exception as e:# (TypeError, ValueError)
        print(f"Error formatting JSON: {e}")

def create_response_format(schema):
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "name": "json_output_schema",
            "schema": {**schema},
            "strict": True
        }
    }
    return response_format

In [112]:
# Replace with your Azure OpenAI details
AZURE_OPENAI_ENDPOINT = ""
API_VERSION = "2024-08-01-preview"
API_KEY = ""
DEPLOYMENT_NAME = "gpt-4o-strict-json"

# Endpoint for creating an assistant
url = f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"

headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY
}

In [113]:
classification_types = ["world_news", "finance", "sports", "politics", "local_news"]
json_output_schema = {
    "type": "object",
    "properties": {
        "response": {
            "type": "object",
            "properties": {
                    "reasoning": { "type": "string" },
                    "classification": { "type": "string", "enum": classification_types}
            },
            "additionalProperties": False,
            "required": ["reasoning", "classification"]
        }
    },
    "additionalProperties": False,
    "required": ["response"]
}
response_format = create_response_format(json_output_schema)

json_output_instance = {
    "response": {
            "reasoning": "This news article is of the type internal_news because of bla bla bla",
            "classification": "internal_news"
    }
}

In [114]:
system_prompt = f"""You are a journalist with over 25 years of experience with great expertise in reading and classifying news articles.
You will receive the content of a news article and your task is to classify it into a set of pre-defined news types.
Please read the entire article in full and then respond in the following format:
{format_json(json_output_instance)}
Add your reasoning to the "reasoning" entry.
Add the classification to the "classification" entry.
"""

In [115]:
user_prompt = """
Kungens tabbe: Sa fel namn på nya prinsessan
Uppdaterad idag 12:20Publicerad idag 11:04
Namnet på prins Carl Philip och Sofias dotter har avslöjats.

Hon ska heta Ines Marie Lilian Silvia och blir hertiginna av Västerbotten.

Men kungen sa ett annat namn under konseljen.

I fredags fick prins Carl Philip och prinsessan Sofia sitt fjärde barn – en dotter.

På måndagen avslöjades namnet på den nya prinsessan under en konselj. Hon ska heta Ines Marie Lilian Silvia, med tilltalsnamnet Ines, skriver hovet i ett pressmeddelande.

Hon blir hertiginna av Västerbotten.

Kungen sa dock ett annat namn under konseljen – och uppgav att prinsessan skulle heta ”Inse”. Senare publicerade hovet ett pressmeddelande med namnet ”Ines Marie Lilian Silvia”.

– Kungen har sagt fel i konseljen i direktsändning, säger hovexperten Johan T Lindwall i SVT:s livesändning.

– Ines är ju ett väldigt, väldigt fint namn. Och det känns som att det har gamla anor. Jag tycker att det är ett otroligt fint tilltalsnamn.

Hovet: Kungen ”jätteledsen” att han sa fel namn
Hovets informationschef Margareta Thorgren säger att kungen är ledsen över att han sa fel tilltalsnamn.

– Kungen är jätteledsen. Han upplevde inte att han sa fel namn när han läste namnen, utan det förstod han senare, säger Margareta Thorgren till SVT.

Thorgren säger att kungen fick reda på namnet kort innan konseljen, och tillägger att det är ”lätt att läsa fel”.

– Det är ofta väldigt hemligt vad kungabarn ska heta ända fram till konseljen, naturligtvis är det också en anledning. I det här fallet vände han på bokstäverna.

”Ljus möter mörker”
Det blir kung Carl Gustaf och drottning Silvias nionde barnbarn.

– Ett barn har kommit till världen, men samtidigt är Sverige i sorg efter illdådet i Örebro förra veckan, sa kungen under konseljen.

– Ljus möter mörker, mörker möter ljus. Låt mig också framföra regeringens varmaste gratulationer, sa statsminister Ulf Kristersson.

Under dagen hålls också tacksägelsegudstjänst, Te Deum, med inbjudna representanter från bland annat riksdagen.

Prinsparet har sedan tidigare tre barn – prins Alexander född 2016, prins Gabriel född 2017 och prins Julian född 2021.

”Vi är lyckliga över att välkomna vår nyfödda dotter till världen. Storebröderna ser nu fram emot att lära känna sin nya lillasyster”, sa prins Carl Philip i ett uttalande efter födseln.
"""


In [116]:
data = {
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    "response_format": response_format
}
response = json.loads(requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"])["response"]
response

{'reasoning': 'This news article is classified as local_news because it focuses on a recent event involving the Swedish royal family, specifically the birth of a new princess and the events surrounding the revelation of her name. The story is centered around a domestic issue related to the Swedish monarchy, including details of a minor mistake made by the king in announcing the name during a council session. These elements are not of international or global interest, nor are they tied to finance, sports, or general politics, which are typical criteria for other classifications like world_news or politics. Instead, it pertains to a ceremonial and cultural matter significant primarily within Sweden.',
 'classification': 'local_news'}